In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.gray();
from matplotlib.pyplot import imshow
import matplotlib.colors as colors


import skimage
from skimage import color, data, filters, restoration, morphology, measure, segmentation
from skimage.io import imread, imsave
from skimage.color import rgb2gray, rgb2hsv
from skimage.transform import rotate, resize
from skimage.transform import AffineTransform, ProjectiveTransform, warp 
from skimage.transform import hough_line, hough_line_peaks
from skimage.filters import gaussian, gabor_kernel, gabor
from skimage.feature import canny, match_template
from skimage.feature import corner_harris, corner_fast, blob_dog, ORB
from skimage.feature import match_descriptors, corner_peaks, plot_matches, corner_subpix
from sklearn.cluster import KMeans, MeanShift
from skimage.measure import ransac
from skimage.segmentation import active_contour



import scipy as sp
from scipy import ndimage
from scipy import signal
from scipy import fft


from time import time

import cv2

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS


from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display

from tqdm.notebook import tqdm

In [ ]:
def filt_func(r, c, sigma = 1):
    return np.exp(-np.hypot(r, c)/sigma) / 2 / np.pi / sigma

# ウィーナフィルタによる画像の復元

In [ ]:
im = rgb2gray(imread('girl.jpg'))


@interact(noise=(0, 1, 0.1), 
          balance=(1, 50, 1),
          sigma=(1, 20, 1))
def g(noise=0.3, sigma=10, balance=10):
    
    fig = plt.figure(figsize=(20, 3))

    fig.add_subplot(1, 4, 1)
    imshow(im)
    plt.axis('off')
    plt.title('original image')


    fig.add_subplot(1, 4, 2)
    impulse = np.zeros((sigma*5, sigma*5)) # use five sigma
    h, w = impulse.shape
    impulse[h//2, w//2] = 1
    psf = gaussian(impulse, sigma=sigma)
    imshow(psf)
    plt.title('PSF: sigma={}'.format(sigma))

    
    fig.add_subplot(1, 4, 3)
    img = signal.fftconvolve(im, psf, mode='same')
    img += noise * img.std() * np.random.standard_normal(img.shape) # 画像をPFSでぼかしてノイズを加える
    imshow(img)
    plt.axis('off')
    plt.title('observed image')
    

    fig.add_subplot(1, 4, 4)
    deconvolved_img = restoration.wiener(img, psf, balance)
    imshow(deconvolved_img, vmin=0, vmax=1)
    plt.axis('off')
    plt.title('restored image, balance {}'.format(balance))

    plt.show()

# HDR合成

In [ ]:
# Axel Jacobs (Photographer) - Axel Jacobs, WebHDR CC BY-SA 2.0
# https://commons.wikimedia.org/wiki/File:HDRI_Sample_Scene_Window_-_01.jpg
# ..
# https://commons.wikimedia.org/wiki/File:HDRI_Sample_Scene_Window_-_12.jpg

urls = \
['https://upload.wikimedia.org/wikipedia/commons/5/51/HDRI_Sample_Scene_Window_-_01.jpg',
'https://upload.wikimedia.org/wikipedia/commons/c/c1/HDRI_Sample_Scene_Window_-_02.jpg',
'https://upload.wikimedia.org/wikipedia/commons/5/5b/HDRI_Sample_Scene_Window_-_03.jpg',
'https://upload.wikimedia.org/wikipedia/commons/6/6e/HDRI_Sample_Scene_Window_-_04.jpg',
'https://upload.wikimedia.org/wikipedia/commons/d/d1/HDRI_Sample_Scene_Window_-_05.jpg',
'https://upload.wikimedia.org/wikipedia/commons/5/51/HDRI_Sample_Scene_Window_-_06.jpg',
'https://upload.wikimedia.org/wikipedia/commons/b/b6/HDRI_Sample_Scene_Window_-_07.jpg',
'https://upload.wikimedia.org/wikipedia/commons/f/f8/HDRI_Sample_Scene_Window_-_08.jpg',
'https://upload.wikimedia.org/wikipedia/commons/8/83/HDRI_Sample_Scene_Window_-_09.jpg',
'https://upload.wikimedia.org/wikipedia/commons/c/c0/HDRI_Sample_Scene_Window_-_10.jpg',
'https://upload.wikimedia.org/wikipedia/commons/6/6e/HDRI_Sample_Scene_Window_-_11.jpg',
'https://upload.wikimedia.org/wikipedia/commons/8/8b/HDRI_Sample_Scene_Window_-_12.jpg']

filenames = []

for i, url in enumerate(urls):
    filename = 'hdr_{:02d}.jpg'.format(i)
    print(url, filename)
#     download(url, filename)
    filenames.append(filename)

In [ ]:
def get_exposure_time(fn):
    # EXIFから露光時間を取り出す
    im = Image.open(fn)
    exif = im._getexif()
    exposure_time = exif[33434]
    exposure_time = exposure_time[0] / exposure_time[1]
    return exposure_time

各画像の露光時間

In [ ]:
fig = plt.figure(figsize=(20, 2))

for i, fn in enumerate(filenames):

    fig.add_subplot(1, 12, i+1)
    im = imread(fn)
    imshow(im)
    plt.axis('off')
    plt.title('{:.03f}'.format(get_exposure_time(fn))) # 露光時間（秒）
plt.show()

In [ ]:
im_list = []
for fn in filenames:
    im = cv2.imread(fn)
    im = cv2.resize(im, (320, 240))
    im_list.append(im)

HDR画像の合成

In [ ]:
merge_mertens = cv2.createMergeMertens()
hdr = merge_mertens.process(im_list)

トーンマッピングによるLDR表示

In [ ]:


@interact(gamma=(0.1, 3, 0.1), saturation=(0.1, 5, 0.1))
def g(gamma=1, saturation=1):

    fig = plt.figure(figsize=(10, 4))


    fig.add_subplot(1, 2, 1)
    tonemap1 = cv2.createTonemap(gamma=gamma)
    ldr1 = tonemap1.process(hdr.copy())
    ldr1 = cv2.cvtColor(ldr1, cv2.COLOR_BGR2RGB)
    imshow(ldr1)
    plt.axis('off')
    plt.title('Tone mappging method 1')

    fig.add_subplot(1, 2, 2)
    tonemap2 = cv2.createTonemapDrago(gamma=gamma, saturation=saturation)
    ldr2 = tonemap2.process(hdr.copy())
    ldr2 = cv2.cvtColor(ldr2, cv2.COLOR_BGR2RGB)
    imshow(ldr2)
    plt.axis('off')
    plt.title('Tone mappging method 2')


    
    plt.show()


# 領域分割

- Felsenszwalb: RGB画像を格子グラフとみなして最小全域木（minimum spanning tree）で分割する手法
https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.felzenszwalb

In [ ]:
img = imread('girl.jpg')

@interact(scale=(10, 150, 10))
def g(scale=50):

    st = time()
    img_seg = segmentation.felzenszwalb(img, scale=scale, sigma=3.0, min_size=20)
    et = time() - st

    fig = plt.figure(figsize=(15,5))

    fig.add_subplot(1, 2, 1)   
    imshow(segmentation.mark_boundaries(img, img_seg, color=(1, 1, 1))) # 領域分割結果を境界線で表示
    plt.axis('off')
    plt.title('segmentation result ({:.2f} sec)'.format(et))

    fig.add_subplot(1, 2, 2)
    imshow(color.label2rgb(img_seg)) # ラベリング結果をカラーで表示．
    plt.axis('off')
    plt.title('segmentation label')

    plt.show()

- slic (simple linear iterative clustering): XYZ色空間と座標をk-meansでクラスタリングしてsuperpixelを作成する手法
https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.slic

In [ ]:
img = imread('girl.jpg')

@interact(n_segments=(10, 300, 10),
          compactness=(1, 30, 1))
def g(n_segments=100, compactness=10):
    
    st = time()
    img_seg = segmentation.slic(img, n_segments=n_segments, compactness=compactness)
    et = time() - st

    fig = plt.figure(figsize=(15,5))

    fig.add_subplot(1, 2, 1)   
    imshow(segmentation.mark_boundaries(img, img_seg, color=(1, 1, 1))) # 領域分割結果を境界線で表示
    plt.axis('off')
    plt.title('segmentation result ({:.2f} sec)'.format(et))

    fig.add_subplot(1, 2, 2)
    imshow(color.label2rgb(img_seg)) # ラベリング結果をカラーで表示．
    plt.axis('off')
    plt.title('segmentation label')


    plt.show()

- quickshift：XYZ色空間と座標をquickshiftでクラスタリングする手法
https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.quickshift

In [ ]:
img = imread('girl.jpg')

@interact(ratio=(0, 1, 0.1),
          kernel_size=(1, 30, 1),
          max_dist=(1, 30, 1))
def g(ratio=100, kernel_size=5, max_dist=10):
    
    st = time()
    img_seg = segmentation.quickshift(img, ratio=ratio, kernel_size=kernel_size, max_dist=max_dist)
    et = time() - st

    fig = plt.figure(figsize=(15,5))

    fig.add_subplot(1, 2, 1)
    imshow(segmentation.mark_boundaries(img, img_seg, color=(1, 1, 1))) # 領域分割結果を境界線で表示
    plt.axis('off')
    plt.title('segmentation result ({:.2f} sec)'.format(et))

    fig.add_subplot(1, 2, 2)
    imshow(color.label2rgb(img_seg)) # ラベリング結果をカラーで表示．
    plt.axis('off')
    plt.title('segmentation label, ratio {0:.1f}, ksize {1}, max_dist {2}'.format(ratio, kernel_size, max_dist))


    plt.show()

- watershed：画像の凹凸を山谷とみなして水の溜まる場所を見つけるwatershedを利用する手法
https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.watershed

In [ ]:
img = imread('girl.jpg')

@interact(markers=(0, 25000, 100),
          compactness=(0, 0.5, 0.1))
def g(markers=5000, compactness=0.1):
    
    st = time()
    markers = markers if markers > 0 else None
    img_seg = segmentation.watershed(img, markers=markers, compactness=compactness)[:, :, 0] # なぜか3チャンネルで返される
    et = time() - st

    fig = plt.figure(figsize=(15,5))

    fig.add_subplot(1, 2, 1)
    imshow(segmentation.mark_boundaries(img, img_seg, color=(1, 1, 1))) # 領域分割結果を境界線で表示
    plt.axis('off')
    plt.title('segmentation result ({:.2f} sec)'.format(et))

    fig.add_subplot(1, 2, 2)
    imshow(color.label2rgb(img_seg)) # ラベリング結果をカラーで表示．
    plt.axis('off')
    plt.title('segmentation label')


    plt.show()

# k-meansクラスタリング

In [ ]:
im = imread('girl.jpg')
im = resize(im, (im.shape[0]//3, im.shape[1]//3))

clustering = KMeans(n_clusters=10)

X = im.reshape((-1, 3))
# clustering.fit(X[::1000, :]) # 画素数を1/1000に間引き
clustering.fit(X)


result = clustering.predict(X)
img_seg = result.reshape(im.shape[:2])

In [ ]:
fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)   
imshow(segmentation.mark_boundaries(im, img_seg, color=(1, 1, 1))) # 領域分割結果を境界線で表示
plt.axis('off')
plt.title('segmentation result')

fig.add_subplot(1, 2, 2)
imshow(color.label2rgb(img_seg)) # ラベリング結果をカラーで表示．
plt.axis('off')
plt.title('segmentation label')

plt.show()

# ガボール特徴も利用したkmeansクラスタリング

In [ ]:
img = rgb2gray(im)

In [ ]:
fig = plt.figure(figsize=(20,9))

n_i = 3
n_j = 10

for j in tqdm(range(n_i)):
    for i in tqdm(range(n_j), leave=False):
        ax = fig.add_subplot(n_i, n_j, i+1 + j*n_j)
        gabor_filter = gabor_kernel(frequency=0.1 * (j+1), bandwidth=1/(2*j+1), theta=0.4 * i).real
        imshow(gabor_filter)
        plt.tight_layout()

plt.show()

In [ ]:
fig = plt.figure(figsize=(20,9))

gabor_features = []

n_i = 3
n_j = 10

for j in tqdm(range(n_i)):
    for i in tqdm(range(n_j), leave=False):
        ax = fig.add_subplot(n_i, n_j, i+1 + j*n_j)
        im_gabor = gabor(img, frequency=0.1 * (j+1), bandwidth=1/(2*j+1), theta=0.4 * i)
        gabor_features.append(im_gabor[0]) # tuble (real, imag)
        imshow(im_gabor[0], cmap="gray")
        plt.axis('off')
        plt.tight_layout()

plt.show()

In [ ]:
gabor_texture = np.array(gabor_features).transpose(1,2,0).reshape((-1, n_i*n_j))

In [ ]:
X = np.hstack((im.reshape((-1, 3)), gabor_texture))

In [ ]:
X.mean(axis=0), X.std(axis=0)

In [ ]:
X -= X.mean(axis=0)
X /= X.std(axis=0)

In [ ]:
X.mean(axis=0), X.std(axis=0)

In [ ]:
clustering = KMeans(n_clusters=10)

# clustering.fit(X[::1000, :]) # 画素数を1/1000に間引き
clustering.fit(X)


result = clustering.predict(X)
img_seg = result.reshape(img.shape[:2])

In [ ]:
fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)   
imshow(segmentation.mark_boundaries(im, img_seg, color=(1, 1, 1))) # 領域分割結果を境界線で表示
plt.axis('off')
plt.title('segmentation result')

fig.add_subplot(1, 2, 2)
imshow(color.label2rgb(img_seg)) # ラベリング結果をカラーで表示．
plt.axis('off')
plt.title('segmentation label')

plt.show()

# 動的輪郭モデル

In [ ]:
img = rgb2gray(imread('coins.jpg'))
imshow(img)
img_s = gaussian(img, 3)

In [ ]:
# #### image boundary
# h, w = img.shape[:3]
# margin = 5
# N = 50
# y = \
# np.concatenate((np.array([margin]*N),
#                 np.linspace(margin, h-1-margin, N),
#                 np.array([h-1-margin]*N),
#                 np.linspace(margin, h-1-margin, N)))
# x = \
# np.concatenate((np.linspace(margin, w-1-margin, N),
#                 np.array([w-1-margin]*N),
#                 np.linspace(w-1-margin, margin, N),
#                 np.array([margin]*N),
#                ))

#### circle
s = np.linspace(0, 2*np.pi, 100)
y = 100 + 80 * np.sin(s)
x = 100 + 80 * np.cos(s)

In [ ]:
snake = np.stack((y, x), axis=1)

all_snake = [snake]
for i in range(100):
    snake = active_contour(img_s,
                           snake,
#                            alpha=0.01, beta=5,
                           alpha=0.02, beta=5,
#                            alpha=0.05, beta=5,
#                            alpha=0.1, beta=5,
                           max_iterations=10,
                           coordinates='rc')
    all_snake.append(snake)

@interact(itr=(0, len(all_snake)-1, 1))
def g(itr=0):
    imshow(img)
    plt.plot(all_snake[0][:, 1], all_snake[0][:, 0], '.w', lw=5)
    plt.plot(all_snake[itr][:, 1], all_snake[itr][:, 0], '.r', lw=5)
    plt.axis('off')
    plt.show()


# テンプレートマッチング

In [ ]:
im = imread('flag.png')[:, :, :3] # remove alpha channel


fig = plt.figure(figsize=(15,5))


fig.add_subplot(1, 2, 1)
imshow(im)
plt.title("original image")


fig.add_subplot(1, 2, 2)
template = im[650:720, 843:960]
imshow(template, interpolation='none')
plt.title('template')


plt.show()

In [ ]:
@interact(angle=(-180, 180, 1))
def g(angle=0):
    
    im_rot = rotate(im, angle=angle, resize=False)

    ncc = match_template(rgb2gray(im_rot), rgb2gray(template))
    x, y = np.unravel_index(np.argmax(ncc), ncc.shape)[::-1] # y,x --> x,y

    fig = plt.figure(figsize=(15,5))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im // 2)

    th, tw, _ = template.shape
    ax.add_patch(plt.Rectangle((x, y), tw, th, edgecolor='r', facecolor='none', lw=3))
    plt.title('detected template as rectangle')


    fig.add_subplot(1, 2, 2)
    imshow(ncc)
    plt.title('NCC')


    plt.show()

# ハフ変換

In [ ]:
im = rgb2gray(imread('card.jpg'))

fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)
imshow(im)
plt.axis('off')
plt.title('original image')

fig.add_subplot(1, 2, 2)
im_edge = canny(im, sigma=3, low_threshold=40/255, high_threshold=100/255)
imshow(im_edge)
plt.axis('off')
plt.title('Canny edge')

plt.show()

In [ ]:
angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
voting_space, theta, rho = hough_line(im_edge, theta=angles)

In [ ]:
imshow(np.log(1 + voting_space),
       extent=[np.rad2deg(theta[-1]), np.rad2deg(theta[0]), rho[-1], rho[0]],
       cmap='gray_r', aspect=0.08)
plt.xlabel('$\\theta$')
plt.ylabel('$\\rho$')
plt.title('voting space')
plt.show()

In [ ]:
_, line_theta, line_rho = hough_line_peaks(voting_space, theta, rho, 
                                           threshold=0.3*voting_space.max(), num_peaks=4)

In [ ]:
x0 = 0
x1 = im.shape[1]
for angle, r in zip(line_theta, line_rho):
    y0 = (r - x0 * np.cos(angle)) / np.sin(angle)
    y1 = (r - x1 * np.cos(angle)) / np.sin(angle)
    plt.plot((x0, x1), (y0, y1), '-r')
imshow(im)
plt.title('lines found by hough transform')
plt.show()

# 特徴点検出

In [ ]:
im = imread('Colosseum.jpg')
img = rgb2gray(im)
imshow(im)
plt.show()

## DoG

In [ ]:
@interact(max_sigma=(10, 300, 10), 
          threshold=(0.02, 1, 0.02))
def g(max_sigma=50, threshold=0.2):
        
    keypoints1 = blob_dog(img, max_sigma=max_sigma, threshold=threshold, overlap=1)
    
    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    for k in keypoints1:
        y, x, s = k
        ax.add_patch(plt.Circle((x, y), s, edgecolor='r', facecolor='none', lw=1))

    plt.title('# keypoints {0}'.format(len(keypoints1)))
    plt.show()

## Fast

In [ ]:
@interact(n=(1, 16, 1), 
          threshold=(0.05, 0.5, 0.01))
def g(n=12, threshold=0.15):

    keypoints1 = corner_peaks(corner_fast(img, n=n, threshold=threshold))

    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    ax.scatter(keypoints1[:, 1], keypoints1[:, 0], color='r', marker='o', s=5)

    plt.title('# keypoints {0}'.format(len(keypoints1)))
    plt.show()

## Harris

In [ ]:
@interact(k=(0, 0.3, 0.01),
          sigma=(0.5, 3, 0.5))
def g(k=0.05, sigma=1):

    keypoints1 = corner_peaks(corner_harris(img, k=k, sigma=sigma))

    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    ax.scatter(keypoints1[:, 1], keypoints1[:, 0], color='r', marker='o', s=5)

    plt.title('# keypoints {0}'.format(len(keypoints1)))
    plt.show()

# GFTT

In [ ]:
@interact(maxCorners=(10,500,10),
          blockSize=(1,20,1))
def g(maxCorners=60, blockSize=3):

    keypoints1 = cv2.goodFeaturesToTrack(img.astype(np.float32),
                                         maxCorners=maxCorners,
                                         qualityLevel=0.01,
                                         minDistance=5,
                                         blockSize=blockSize)
    keypoints1 = np.squeeze(keypoints1)


    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    ax.scatter(keypoints1[:, 0], keypoints1[:, 1], color='r', marker='o', s=5)


    plt.title('# keypoints {0} {1}'.format(len(keypoints1), blockSize))
    plt.show()

# AKAZE

In [ ]:
@interact(descriptor_type=[3,5],
          threshold=(0.0001, 0.005, 0.0001))
def g(threshold=0.001, descriptor_type=3):

    detector = cv2.AKAZE_create(descriptor_type=descriptor_type, threshold=threshold)
    keypoints1, descriptors1 = detector.detectAndCompute(im, None)


    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    for k in keypoints1:
        x, y = k.pt
        s = k.size
        ax.add_patch(plt.Circle((x, y), s, edgecolor='r', facecolor='none', lw=1))


    plt.title('# keypoints {0}'.format(len(keypoints1)))
    plt.show()

# BRISK

In [ ]:
@interact(thresh=(10, 200, 10))
def g(thresh=100):

    detector = cv2.BRISK_create(thresh=thresh)
    keypoints1, descriptors1 = detector.detectAndCompute(im, None)


    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    for k in keypoints1:
        x, y = k.pt
        s = k.size
        ax.add_patch(plt.Circle((x, y), s, edgecolor='r', facecolor='none', lw=1))


    plt.title('# keypoints {0}'.format(len(keypoints1)))
    plt.show()

## ORB

In [ ]:
@interact(n_keypoints=(100, 2000, 100))
def g(n_keypoints=1000):

    descriptor_extractor = ORB(n_keypoints=n_keypoints)
    descriptor_extractor.detect_and_extract(img)
    keypoints1 = descriptor_extractor.keypoints # 特徴点の(y,x)座標
    # descriptors1 = descriptor_extractor.descriptors # 特徴量ベクトル

    fig = plt.figure(figsize=(25,10))

    ax = fig.add_subplot(1, 2, 1)

    imshow(im)

    ax.scatter(keypoints1[:, 1], keypoints1[:, 0], color='r', marker='o', s=5)

    plt.title('# keypoints {0}'.format(len(keypoints1)))
    plt.show()

# パノラマ画像作成

In [ ]:
img1 = imread('Colosseum.jpg')
img2 = img1.copy()

img1 = img1[:250, :300]
img1 = rotate(img1, angle=45, resize=True)

img2 = img2[100:, 200:600]
img2 = rotate(img2, angle=-5, resize=True)

fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)
imshow(img1)

fig.add_subplot(1, 2, 2)
imshow(img2)

plt.show()

In [ ]:
# # 実画像を重ね合わせてみるための画像の例はこちら．

# # Peter Haas CC BY-SA 3.0
# # https://commons.wikimedia.org/wiki/File:Notre-Dame_de_Paris_2013-07-24.jpg
# img1 = imread('https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Notre-Dame_de_Paris_2013-07-24.jpg/355px-Notre-Dame_de_Paris_2013-07-24.jpg')/255

# # Dietmar Rabich CC BY-SA 4.0
# # https://commons.wikimedia.org/wiki/File:Paris,_Notre_Dame_--_2014_--_1445.jpg
# img2 = imread('https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/Paris%2C_Notre_Dame_--_2014_--_1445.jpg/301px-Paris%2C_Notre_Dame_--_2014_--_1445.jpg')/255

画像周囲を黒画素で拡張しておきます（後の処理を簡単にするため）

In [ ]:
def padding(im, pad=200):
    h, w = im.shape[:2]
    im_pad = np.zeros((h+2*pad, w+2*pad, 3))
    im_pad[pad:pad+h, pad:pad+w] = im
    return im_pad
    
img1 = padding(img1)
img2 = padding(img2)

fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)
imshow(img1)

fig.add_subplot(1, 2, 2)
imshow(img2)

plt.show()

In [ ]:
# カラー画像はグレースケールに変換
img1g = rgb2gray(img1)
img2g = rgb2gray(img2)

scikit-imageのORB特徴量を使います

In [ ]:
# ORB特徴量を使います
descriptor_extractor = ORB(n_keypoints=1000)

# 画像1から特徴を検出
descriptor_extractor.detect_and_extract(img1g)
keypoints1 = descriptor_extractor.keypoints # 特徴点の(y,x)座標
descriptors1 = descriptor_extractor.descriptors # 特徴量ベクトル

# 画像2から特徴を検出
descriptor_extractor.detect_and_extract(img2g)
keypoints2 = descriptor_extractor.keypoints # 特徴点の(y,x)座標
descriptors2 = descriptor_extractor.descriptors # 特徴量ベクトル

別の特徴量を使うときには以下のコードをコメントアウトして使ってください

In [ ]:
# # BRISK特徴量を使います
# detector = cv2.BRISK_create()

# # AKAZE特徴量を使います
# detector = cv2.AKAZE_create()

# # 画像1から特徴を検出
# keypoints1, descriptors1 = detector.detectAndCompute((img1*255).astype(np.uint8), None)
# keypoints1 = np.array([(k.pt[1], k.pt[0]) for k in keypoints1]) # (x,y) --> (y,x)

# # 画像2から特徴を検出
# keypoints2, descriptors2 = detector.detectAndCompute((img2*255).astype(np.uint8), None)
# keypoints2 = np.array([(k.pt[1], k.pt[0]) for k in keypoints2]) # (x,y) --> (y,x)

In [ ]:
# 特徴量のマッチングをします
matches12 = match_descriptors(descriptors1, descriptors2, cross_check=True)

In [ ]:
# マッチング結果を表示します．対応する点が線で結ばれています．間違っているのもありますね．
plt.figure(figsize=(15,10))
ax = plt.axes()
plot_matches(ax,
             img1, img2, 
             keypoints1, keypoints2, 
             matches12)
ax.axis('off')
# ax.set_title('Correspondences')
plt.show()

RANSACという方法で，間違っている対応（誤対応）を除去します．

http://scikit-image.org/docs/dev/auto_examples/plot_matching.html

In [ ]:
# 対応点をsrcとdstに入れます
src = [] # img1
dst = [] # img2
for coord in matches12:
    src.append( keypoints1[coord[0]] )
    dst.append( keypoints2[coord[1]] )
src = np.array(src)
dst = np.array(dst) 

In [ ]:
# RANSAC実行
model_robust, inliers = ransac((src, dst), 
#                                AffineTransform, min_samples=3,
                               ProjectiveTransform, min_samples=4,
                               residual_threshold=2, 
                               max_trials=2000)
inlier_idxs = np.nonzero(inliers)[0] # 正しい対応（インライア）

RANSAC後の対応点を見てみよう．誤対応が少なくなっていますね

In [ ]:
plt.figure(figsize=(15,10))
ax = plt.axes()
plot_matches(ax, 
             img1, img2,
             src, dst,
             np.column_stack((inlier_idxs, inlier_idxs)) )
ax.axis('off')
# ax.set_title('Correct correspondences')
plt.show()

In [ ]:
# それではあらためて，正しい対応（インライア）だけを使って変換パラメータを推定しなおします．

#src = src[inliers] # インライアだけを取り出す
#src = src[:, [1,0] ] # (y,x)座標ベクトルを(x,y)座標ベクトルに変換する
#
#dst = dst[inliers]
#dst = dst[:, [1,0] ]
#
#model_robust.estimate( src, dst )


#上と同じことを1行でやってます
model_robust.estimate( src[inliers][:, [1,0] ], dst[inliers][:, [1,0] ] )


#注釈：
# keypoint, ransac, plot_matchは(y,x)座標ベクトルを使っていますが，
# warpは(x,y)座標ベクトルを使います．だから，ここでxyを変化して推定しなおしています

推定された変換パラメータはこれ

In [ ]:
if type(model_robust) == AffineTransform:
    print("scale: ", model_robust.scale)
    print("translation [pixels]: ", model_robust.translation)
    print("rotaiton [radians]: ", model_robust.rotation)
print("Transform matrix:")
print(model_robust.params)

それでは画像2と1を合成しましょう．

In [ ]:
fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)
img_warped1 = warp( img1, model_robust.inverse )
imshow(img_warped1)

fig.add_subplot(1, 2, 2)
img_warped2 = img2
imshow(img_warped2)

plt.show()

2枚の画像のサイズが違うので同じサイズにします

In [ ]:
img_pano = np.zeros((max(img_warped1.shape[0], img_warped2.shape[0]),
                     max(img_warped1.shape[1], img_warped2.shape[1]), 3), dtype=np.float)

img_pano1 = img_pano.copy()
img_pano1[:img_warped1.shape[0], :img_warped1.shape[1]] = img_warped1

img_pano2 = img_pano.copy()
img_pano2[:img_warped2.shape[0], :img_warped2.shape[1]] = img_warped2


fig = plt.figure(figsize=(15,5))

fig.add_subplot(1, 2, 1)
imshow(img_pano1)

fig.add_subplot(1, 2, 2)
imshow(img_pano2)

plt.show()

足してみよう．パノラマができたかな？

In [ ]:
fig = plt.figure(figsize=(15,15))

img_pano = (img_pano1 + img_pano2) / 2
imshow( img_pano )
plt.axis('off')
plt.show()

画像の足し算ではいまいち．各画素のmaxをとってみよう．

In [ ]:
fig = plt.figure(figsize=(15,15))

img_pano = np.maximum(img_pano1 , img_pano2)
imshow( img_pano )
plt.axis('off')
plt.show()